# FUNCIONES

In [23]:
import os
import numpy as np
import copy
from math import sqrt
from scipy.linalg import solve_triangular

### Creación matrices

In [24]:
def crea_matriz(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias
                            
    params: renglones       no. de renglones de la matriz
            columnas        no. de renglones de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

### Factorización QR

In [25]:
def house(x):
    """
    Función que calcula la proyección de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    constante utilizada para obtener v
            v       vector que representa la reflexión de householder
    """
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

In [26]:
def factorizacion_QR(A):
    """
    Función que genera una matriz que contendrá información escencial de las proyecciones householder
    (vectores v's) y componentes de la matriz triangular superior R, del estilo:
    [r11      r12      r13      r14    ]
    [v_2_(1)  r22      r23      r24    ]
    [v_3_(1)  v_3_(2)  r33      r34    ]
    [v_4_(1)  v_4_(2)  v_4_(3)  r44    ]
    [v_5_(1)  v_5_(2)  v_5_(3)  v_5_(4)]
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: A_r_v  Matriz (mxn) con la información escencial (es igual a la matriz R, pero en lugar de tener ceros
                   en la parte inferior, contiene info de los vectores householder que serán útiles para
                   futuros cálculos, que entre otros están el calcular la matriz ortonormal Q)
    """
    m=A.shape[0]
    n=A.shape[1]
    A_r_v=copy.copy(A)
    for j in range(n):
        beta, v=house(A_r_v[j:m,j])
        A_r_v[j:m,j:n]=A_r_v[j:m,j:n]-beta*(np.outer(v,v)@A_r_v[j:m,j:n])
        A_r_v[(j+1):m,j]=v[1:(m-j)]
    return A_r_v

In [27]:
def QT_C(A_r_v,C):
    """
    Función que calcula el producto matricial de Q_transpuesta por una matriz dada C
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: M       Matriz con numero al azar
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    QT_por_C=np.eye(m)
    for j in range(n-1,-1,-1):
        v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
        beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
        QT_por_C[j:m,j:m]=C[j:m,j:m]-beta*np.outer(v,v)@C[j:m,j:m]
    return QT_por_C

In [28]:
def Q_j(A_r_v,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van obteniendo n Qj's,
    que si se multiplican todas da por resultado Q=Q1*Q2*...*Qn)
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: Qj      Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Qj=np.eye(m)
    v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
    beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
    Qj[j:m,j:m]=np.eye(m-j)-beta*np.outer(v,v)
    return Qj

### Funciones para solución de Sistemas de Ecuaciones Lineales

In [29]:
def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """
    A_r_v=factorizacion_QR(A)
    m=A_r_v.shape[0]
    #Q=np.transpose(QT_C(A_r_v,np.eye(m)))
    #R=np.transpose(Q)@A
    n=A_r_v.shape[0]
    Q=np.eye(m)
    R=copy.copy(A)
    for j in range(m):
        Qj=Q_j(A_r_v,j)
        Q=Q@Qj
        R=Q_j(A_r_v,j)@R
    b_prima=np.transpose(Q)@b
    x = solve_triangular(R, np.transpose(Q)@b)
    return x

#### Eliminación por bloques

In [30]:
def bloques(A, b=False, n1=False, n2=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            n2  Numero de renglones que tendrá el 2do bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    # Primero definimos el n
    m,n = A.shape

    # Condiciones de A
    # Si no se dan los n deseados, se intentan hacer los bloques casi iguales
    if  not (n1&n2):
        n1 = n//2
        n2 = n - n1
    # Los bloques deben cumplir la condicion de tamaño
    elif n1+n1 != n:
        sys.exit('n1 + n2 debe ser igual a n')
    else:
        None

    # Condiciones de b
    if  b is False:
        b1 = None
        b2 = None
        print('condicion1')
    elif len(b) == m:
        b1 = b[:n1]
        b2 = b[n1:m]
    else:
        sys.exit('los renglones de A y b deben ser del mismo tamaño')

    A11 = A[:n1,:n1]
    A12 = A[:n1,n1:n]
    A21 = A[n1:m,:n1]
    A22 = A[n1:m,n1:n]

    return A11,A12,A21,A22,b1,b2

In [31]:
def eliminacion_bloques(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    if np.linalg.det(A)==0:
        sys.exit('A debe ser no singular')

    A11,A12,A21,A22,b1,b2 = bloques(A,b)

    if np.linalg.det(A11)==0:
        sys.exit('A11 debe ser no singular')

    ## 1. Calcular A11^{-1}A12 y A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    ## Aquí se debe usar el método QR una vez que esté desarrollado

    ## Definimos y = A11^{-1}b1, por tanto A11y=b1. Resolviendo el sistema anterior para 11y:
    y = Solucion_SEL_QR_nxn(A11,b1)
    #y = np.linalg.solve(A11,b1)

    ## Definimos Y = A11^{-1}A12
    Y = Solucion_SEL_QR_nxn(A11,A12)
    #Y = np.linalg.solve(A11,A12)

    ## 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
    S = A22 - A21@Y
    b_h = b2 - A21@y

    ## 3. Resolver Sx2 = b_hat
    x2 = Solucion_SEL_QR_nxn(S,b_h)
    #x2 = np.linalg.solve(S,b_h)

    ## 4. Resolver A11x1 = b1-A12X2
    x1 = Solucion_SEL_QR_nxn(A11,b1-A12@x2)
    #x1 = np.linalg.solve(A11,b1-A12@x2)

    return np.concatenate((x1,x2), axis=0)

# EJEMPLOS

**Generación de matrices**

In [32]:
#Prueba también definiendo una matriz entrada por entrada, solo asegurate que sus entradas sean
#tipo flotantes/dobles (pues si son de tipo enteras arrastrarán errores de redondeo considerables)
m=3
n=3
A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')
A

array([[1., 2., 1.],
       [2., 3., 2.],
       [1., 2., 3.]])

In [33]:
#Generar una matriz aleatoria de tamaño definido (renglones y columnas) y valores aleatorios comprendidos en un rango, pudiendo ser sus entradas de tipo enteros o dobles
m=5
n=3
A=np.round(crea_matriz(m,n,6,-6,False),2)
A

array([[ 5.88, -5.89,  5.73],
       [ 5.15, -4.67, -0.53],
       [ 0.72,  1.92,  3.08],
       [ 3.61,  3.1 , -2.42],
       [ 4.36, -4.51,  1.88]])

**Ejemplos de factorización de una matriz A**

In [34]:
A_r_v = factorizacion_QR(A)
np.round(A_r_v,4)

array([[ 9.6777, -6.7965,  3.3728],
       [-1.3561,  6.6291, -2.7784],
       [-0.1896, -0.2077,  5.7355],
       [-0.9506, -0.3934, -1.8217],
       [-1.1481,  0.3445, -2.5155]])

In [35]:
#Obtención de Q con la función QT_C
Q=np.transpose(QT_C(A_r_v,np.eye(m)))
np.round(Q,4)

array([[ 0.6076,  0.5322,  0.0744,  0.373 ,  0.4505],
       [ 0.5322,  0.2783, -0.1009, -0.5059, -0.611 ],
       [ 0.0744, -0.1009,  0.9859, -0.0707, -0.0854],
       [ 0.373 , -0.5059, -0.0707,  0.6454, -0.4283],
       [ 0.4505, -0.611 , -0.0854, -0.4283,  0.4828]])

In [36]:
#Así obtenemos R
R=np.transpose(Q)@A
np.round(R,4)

array([[ 9.6777, -6.7965,  3.3728],
       [ 0.    , -3.4407,  2.6666],
       [ 0.    ,  2.0919,  3.5269],
       [ 0.    ,  3.9617, -0.1793],
       [ 0.    , -3.4693,  4.5862]])

In [37]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
Q=np.eye(m)
R=copy.copy(A)
for j in range(n):
    Qj=Q_j(A_r_v,j)
    print('\nQ',j,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=Q_j(A_r_v,j)@R
    print('\nAplicando las Qjs a A por la izquierda para obtener R, iteracion ',j,':\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))


Q 0 :
 [[ 0.6076  0.5322  0.0744  0.373   0.4505]
 [ 0.5322  0.2783 -0.1009 -0.5059 -0.611 ]
 [ 0.0744 -0.1009  0.9859 -0.0707 -0.0854]
 [ 0.373  -0.5059 -0.0707  0.6454 -0.4283]
 [ 0.4505 -0.611  -0.0854 -0.4283  0.4828]]

Aplicando las Qjs a A por la izquierda para obtener R, iteracion  0 :
 [[ 9.6777 -6.7965  3.3728]
 [ 0.     -3.4407  2.6666]
 [ 0.      2.0919  3.5269]
 [ 0.      3.9617 -0.1793]
 [ 0.     -3.4693  4.5862]]

Q 1 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.     -0.519   0.3156  0.5976 -0.5233]
 [ 0.      0.3156  0.9344 -0.1241  0.1087]
 [ 0.      0.5976 -0.1241  0.7649  0.2059]
 [ 0.     -0.5233  0.1087  0.2059  0.8197]]

Aplicando las Qjs a A por la izquierda para obtener R, iteracion  1 :
 [[ 9.6777 -6.7965  3.3728]
 [-0.      6.6291 -2.7784]
 [ 0.      0.      4.658 ]
 [ 0.     -0.      1.9628]
 [ 0.      0.      2.7103]]

Q 2 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.    ]
 [ 0.      0.      0.8121  0.3422  0.4726]

In [38]:
#Se prueba un ejemplo más grande con 10^4 entradas
m=125
n=80
A=np.round(crea_matriz(m,n,10,-10,False),2)
A_r_v = factorizacion_QR(A)
A_r_v

array([[ 6.33213250e+01,  2.98206015e+00, -4.78701291e-01, ...,
         4.75119401e+00, -4.25444351e-01,  4.96567310e+00],
       [-6.16510221e-04,  6.41135946e+01,  3.93309746e+00, ...,
         1.43879242e+00, -3.47087697e+00, -1.55807105e+00],
       [-1.15133284e-01, -1.31427912e-01,  6.60921099e+01, ...,
        -4.83569383e+00,  4.49658419e+00,  9.50941830e+00],
       ...,
       [ 4.36180981e-02,  1.36084171e-01,  4.67173867e-02, ...,
        -2.17457044e-03,  1.88677662e-01, -1.11831885e-01],
       [-1.19294728e-01,  1.22287780e-01,  5.30385014e-02, ...,
         1.74975954e-01, -2.29575016e-01,  1.74943207e-01],
       [-1.60292657e-02,  1.34846071e-02, -1.47976041e-01, ...,
         8.45097376e-02, -1.37344965e-01,  2.20156191e-01]])

**Solucion de SEL**

### A) Sistema con soluciones únicas

In [39]:
#Generamos un sistema de ecuaciones lineales
m=6
n=6
A=np.round(crea_matriz(m,n,6,-6,False),2)
b = np.round(crea_matriz(m,1,6,-6,False),2)
print("A:",A)
print("b:",b)

A: [[-3.03 -2.23  5.52 -0.13 -0.    1.93]
 [ 5.04  3.7   0.6   2.71  0.98  5.92]
 [ 5.71  4.59  2.06  0.93 -4.95  0.91]
 [ 1.16  4.03  2.58 -2.25  4.81  2.28]
 [ 2.45  3.48  4.95  1.57  5.2   1.27]
 [ 5.65 -1.22  5.19 -3.03  3.83  0.18]]
b: [[ 2.08]
 [ 2.55]
 [-5.59]
 [-4.79]
 [ 4.9 ]
 [ 4.87]]


##### Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [40]:
np.linalg.solve(A,b)

array([[ 0.66745352],
       [-1.50599918],
       [ 0.24512067],
       [ 1.7983445 ],
       [ 0.90688667],
       [-0.19444319]])

##### Ahora usemos la factorización QR

In [41]:
x=Solucion_SEL_QR_nxn(A,b)
x

array([[ 0.66745352],
       [-1.50599918],
       [ 0.24512067],
       [ 1.7983445 ],
       [ 0.90688667],
       [-0.19444319]])

In [42]:
#Checamos que sea solucion (debe ser igual al vector b)
A@x

array([[ 2.08],
       [ 2.55],
       [-5.59],
       [-4.79],
       [ 4.9 ],
       [ 4.87]])

##### Por último usemos eliminación por bloques con QR

In [43]:
eliminacion_bloques(A,b)

array([[ 0.66745352],
       [-1.50599918],
       [ 0.24512067],
       [ 1.7983445 ],
       [ 0.90688667],
       [-0.19444319]])

### B) Sistema sin solución

### C) Sistema con infinitas soluciones